In [ ]:
# Hàm để đặt một lệnh mua
from binance.client import Client
from binance.enums import *
import os
# Đặt API key và Secret key
api_key = 'kMYSAWRigplgKRg16aZONgTs5lRXroWbd33hblLAXDmpT1ddXLXLJJZnbVbE03GY' # Future
api_secret = 'PjsgJVWq5Qy5XHORgXSpzIwQFLxeK15Ja9auiw5TiYCs0vskG93uXO3F3bXxJz18' # Future

def orderSendFuture(data):
    # Lấy dữ liệu từ từ điển
    datetime = data['Datetime']
    open_price = data['Open']
    close_price = data['Close']
    volume = data['Volume']
    sma = data['SMA']
    short_ema = data['short_ema']
    long_ema = data['long_ema']
    macd = data['MACD']
    signal_line = data['Signal_Line']
    buy_signal = data['Buy_Signal']
    
    sell_signal = data['Sell_Signal']
    symbol = data['Symbol']
    insertdate = data['Insertdate']
    
    # Khởi tạo client
    client = Client(api_key, api_secret)

    quantity = 5
    leverage = 40  # Đòn bẩy

    # Thay đổi đòn bẩy
    client.futures_change_leverage(symbol=symbol, leverage=leverage)
    
    # Thay đổi loại ký quỹ chéo (Cross) sang ký quỹ riêng lẻ (ISOLATED)
    # client.futures_change_margin_type(symbol=symbol, marginType='ISOLATED')

    # Lấy thông tin giá gần nhất của ETHUSDT
    current_price = client.get_symbol_ticker(symbol=symbol)
    entry_price = current_price["price"]

    # Chuyển đổi entry_price sang float
    entry_price = float(entry_price)
    
    allowed_precision = 3  # Giả sử độ chính xác tối đa là 6 chữ số thập phân

    order = None

    try:
        if buy_signal == 'True':
            print("Placing a BUY market order:", quantity)
            # Đặt Take Profit và Stop Loss (cần tính toán dựa trên chiến lược)
            take_profit = round(entry_price * 1.01, allowed_precision)
            stop_loss = round(entry_price * 0.98, allowed_precision)
            
            # Đặt lệnh mua
            buy_order = client.futures_create_order(
                symbol=symbol,
                side=SIDE_BUY,
                type=ORDER_TYPE_MARKET,
                quantity=quantity
            )           
            print("Buy order placed successfully:", buy_order)
            
            print("Placing a TAKE PROFIT market order.")
            tp_order = client.futures_create_order(
                symbol=symbol,
                side=SIDE_SELL,
                type=FUTURE_ORDER_TYPE_TAKE_PROFIT_MARKET ,
                timeInForce=TIME_IN_FORCE_GTC,
                quantity=quantity,
                stopPrice=take_profit,
                reduceOnly='true'
            )
            print("Take profit order placed successfully:", tp_order)

            print("Placing a STOP LOSS stop market order.")
            sl_order = client.futures_create_order(
                symbol=symbol,
                side=SIDE_SELL,
                type=FUTURE_ORDER_TYPE_STOP_MARKET,
                quantity=quantity,
                stopPrice=stop_loss,
                reduceOnly='true'
            )
            print("Stop loss order placed successfully:", sl_order)

        elif sell_signal == 'True':
            print("Placing a SELL market order.")
            # Đặt Take Profit và Stop Loss (cần tính toán dựa trên chiến lược)
            take_profit = round(entry_price * 0.99, allowed_precision)
            stop_loss = round(entry_price * 1.02, allowed_precision)
            
            # Đặt lệnh bán
            sell_order = client.futures_create_order(
                symbol=symbol,
                side=SIDE_SELL,
                type=ORDER_TYPE_MARKET,
                quantity=quantity
            )            
            print("Sell order placed successfully:", sell_order)
           
            print("Placing a TAKE PROFIT market order.")
            tp_order = client.futures_create_order(
                symbol=symbol,
                side=SIDE_BUY,
                type=FUTURE_ORDER_TYPE_TAKE_PROFIT_MARKET,
                timeInForce=TIME_IN_FORCE_GTC,
                quantity=quantity,
                stopPrice=take_profit,
                reduceOnly='true'
            )
            print("Take profit order placed successfully:", tp_order)

            print("Placing a STOP LOSS stop market order.")
            sl_order = client.futures_create_order(
                symbol=symbol,
                side=SIDE_BUY,
                type=FUTURE_ORDER_TYPE_STOP_MARKET,
                quantity=quantity,
                stopPrice=stop_loss,
                reduceOnly='true'
            )
            print("Stop loss order placed successfully:", sl_order)

    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
import redis

def entryFuture(hash):
    # Xử lý dữ liệu ở đây
    print("Dữ liệu đã được xử lý")

    # Tạo kết nối Redis
    r = redis.Redis(host='localhost', port=6379, db=0)

    # Hash key
    hash_key = hash

    # Đọc dữ liệu từ hash
    data = r.hgetall(hash_key)
    
    if data:
        # Chuyển đổi dữ liệu từ bytes sang chuỗi nếu cần
        data = {key.decode('utf-8'): value.decode('utf-8') for key, value in data.items()}
        # Xử lý dữ liệu
        orderSendFuture(data)
        # Xóa hash sau khi xử lý
        r.delete(hash_key)
        print(f"Hash '{hash_key}' đã được xóa khỏi Redis.")
    else:
        print(f"Không tìm thấy dữ liệu cho hash '{hash_key}'.")

In [3]:
from datetime import datetime, timedelta
import schedule
import time

def schedule_entryFuture():
    entryFuture('OG_CRTO_FT_MACD dua tren HA')

# Lên lịch hàm scan_market để chạy mỗi 15 phút
schedule.every(10).seconds.do(schedule_entryFuture)

while True:
    schedule.run_pending()  # Hàm này được gọi liên tục để kiểm tra xem có công việc nào đã đến thời gian cần thực hiện hay không
    time.sleep(1)

Dữ liệu đã được xử lý
Không tìm thấy dữ liệu cho hash 'OG_CRTO_FT_MACD dua tren HA'.
Dữ liệu đã được xử lý
Không tìm thấy dữ liệu cho hash 'OG_CRTO_FT_MACD dua tren HA'.
Dữ liệu đã được xử lý
Không tìm thấy dữ liệu cho hash 'OG_CRTO_FT_MACD dua tren HA'.
Dữ liệu đã được xử lý
Không tìm thấy dữ liệu cho hash 'OG_CRTO_FT_MACD dua tren HA'.
Dữ liệu đã được xử lý
Không tìm thấy dữ liệu cho hash 'OG_CRTO_FT_MACD dua tren HA'.
Dữ liệu đã được xử lý
Không tìm thấy dữ liệu cho hash 'OG_CRTO_FT_MACD dua tren HA'.
Dữ liệu đã được xử lý
Không tìm thấy dữ liệu cho hash 'OG_CRTO_FT_MACD dua tren HA'.
Dữ liệu đã được xử lý
Không tìm thấy dữ liệu cho hash 'OG_CRTO_FT_MACD dua tren HA'.
Dữ liệu đã được xử lý
Không tìm thấy dữ liệu cho hash 'OG_CRTO_FT_MACD dua tren HA'.
Dữ liệu đã được xử lý
Không tìm thấy dữ liệu cho hash 'OG_CRTO_FT_MACD dua tren HA'.
Dữ liệu đã được xử lý
Không tìm thấy dữ liệu cho hash 'OG_CRTO_FT_MACD dua tren HA'.
Dữ liệu đã được xử lý
Không tìm thấy dữ liệu cho hash 'OG_CRTO_FT